In [2]:
# Uses code from 'make j from scratch'
D.<x> = PolynomialRing(QQ)
R.<x> = PolynomialRing(QQ)
S = PowerSeriesRing(QQ,'x', default_prec=300)
L=LaurentSeriesRing(QQ,'x', default_prec=300)
#S.<x,y>=PowerSeriesRing(QQ,default_prec=300)
# Using "R = PolynomialRing(QQ,'x')" produces an error.
# Yet using S as defined here is fine (see definition of Fraleigh4 below)
# But S = PowerSeriesRing(QQ,'x','y') also produces an error.
def multiplicative_order(d,n):
    for k in [0..n]:
        if (mod(n,d^k)==0)&(mod(n,d*d^k)!=0):return k
    return -1
def pochhammer(x,n):
    return product ([x+k for k in [0..n-1]])
def cRaleigh2(alpha,beta,nu):
    pchhmrA = pochhammer(alpha,nu)
    pchhmrB = pochhammer(beta,nu)
    fnu=factorial(nu)
    answer = D(pchhmrA*pchhmrB/fnu^2)
    return answer
def term(alpha,beta,p):
    return D(1/(alpha + p) + 1/(beta + p) - 2/(1 + p))
def eRaleigh(alpha, beta, nu):
    return D(sum([term(alpha, beta, p) for p in [0..nu - 1]]))
def FstarRaleigh2(alpha,beta,u,terms):
    fsr = sum([cRaleigh2(alpha, beta, nu)*eRaleigh(alpha,beta,nu)*u^nu for nu in [1..terms]])
    return D(fsr)
def Fraleigh2(alpha,beta,u,terms): 
    return D(sum([cRaleigh2(alpha,beta,nu)*u^nu for nu in [0..terms]]))
def FstarRaleigh3(n,m,x):
    alpha = (1/2-1/m)/2
    beta = (1/2+1/m)/2
    fsr2 = FstarRaleigh2(alpha,beta,x,n)
    return D(fsr2)
def Fraleigh4(n,m,x):
    alpha = (1/2-1/m)/2
    beta = (1/2+1/m)/2
    fr2 = Fraleigh2(alpha,beta,x,n)
    return D(fr2)
def exNo3c(n,m):
    a1 = S(x)*exp(S(FstarRaleigh3(n,m,x)/Fraleigh4(n,m,x))).O(2*n)
    # S(x) instead of x in df of a1 to avoid an error message
    # after use of the stripQuotationMarks command below.
    a2 = S(a1).O(n+1)
    return a2 # applying S to argument of exp is key.
def J(n,m):
    exn = exNo3c(n+2,m)
    jnm = 1/exn.reverse()
    return jnm.O(n+1) # Jtest is faster than J, and they agree.
def normalJ(n,m):
    f = J(n,m)
    return f.truncate(n+1)
def Jcoefficient(n,m):
    f = J(n+2,m)
    f1 = f.truncate(n+1)
    f1=f1.list()
    f1=Set(f1)
    g1 = f.truncate(n)
    g1 = g1.list()
    g1 = set(g1)
    h = f1.difference(g1)
    h = h.list()
    return h[0]
def xjNew(n,m):
    def xjNewStep(n,m):
        f=J(n,m)
        fl=f.list()
        rt=0
        for i in [0..len(fl)-1]:
            rt=rt+fl[i]*x**i*(m**3*2**6)**i
        return rt
    return xjNewStep(n-1,m)
def j(n,m):return L(xjNew(n,m)/x)


    

def polynomialCoefficient(n,poly):
    return poly.list()[n]
def leadingCoefficient(polynomial):
    dg=polynomial.degree()
    return polynomialCoefficient(dg,polynomial)
def factorpairToFactor(factorpair):
    (monomial,exponent)=factorpair
    return monomial^exponent
def numericalfactor(polynomial):
    fp=polynomial.factor()
    lst=[factorpairToFactor(factorpair) for factorpair in fp]
    return polynomial/product(lst)
def firstfactor(polynomial):
    fp=polynomial.factor()
    fp=FiniteEnumeratedSet(fp)
    return factorpairToFactor(fp.first())
def lastfactor(polynomial):
    fp=polynomial.factor()
    fp=FiniteEnumeratedSet(fp)
    return factorpairToFactor(fp.last())
def reduced(polynomial):
    return numericalfactor(polynomial)*lastfactor(polynomial)
def stripQuotationMarks(string):
    var('x')
    return sage_eval(string,locals={'x':x})
def sigmaOdd(n):
    sum=0
    dvs=divisors(n)
    lnth=len(dvs)
    for k in [0..lnth-1]:
        dv=dvs[k]
        if mod(dv,2)==1:
            sum=sum+dv
    return sum

def division(dividend, divisor) : 
    quotient=(dividend._maxima_().divide(divisor).sage())[0] 
    remainder=(dividend._maxima_().divide(divisor).sage())[1] 
    return (quotient,remainder)

def HsubI(n,m):
    jay = J(n+2,m)
    numerator = x^m*derivative(jay,x)^m
    denominator = jay^(m-1)*(jay-1)
    power=1/(m-2)
    return -L((numerator/denominator)^power).truncate(n+1)

def HsubItoMminus2(n,m):
    jay = J(n+2,m)
    numerator = x^m*derivative(jay,x)^m
    denominator = jay^(m-1)*(jay-1)
    return (-1)^m*L((numerator/denominator)).truncate(n+1)

def HsubItoMminus2Strike(n,m):
    answer=HsubItoMminus2(n,m)
    al=answer.list()
    dg=answer.degree()
    sm=sum(al[k]*(2^6*m^3*x)^k for k in [0..dg])*(-1)^m
    return sm*(-1)^m

def H6(n,m):
    def base6(n,m):
        jay=J(n+1,m)
        num=(x*derivative(jay,x))^m # chain rule; variable is tau not x.
        den = expand(jay^(m-1)*(jay-1))
        return L((-1)^m*(num/den)).O(n+1)
    answer=S(base6(n,m)^(1/(m-2)))
    return answer.polynomial()
     #if argument is an S type then 'polynomial' takes no argument.
    
def H6strike(n,m): 
    poly=H6(n,m)
    polylist=poly.list()
    polydegree=poly.degree()
    answer=sum(polylist[k]*(x*m^3*2^6)^k for k in [0..polydegree])
    return answer

def polynomialExponentList(polynomial):
    pel=[]
    fp=polynomial.factor()
    lf=len(fp)
    for n in [0..lf-1]:
        part = fp[n]
        (base,exponent) = part
        pel=pel+[exponent]
    return pel

print("ok")

ok


In [4]:
print(HsubI(5,3))

1 - 7/24*x - 77/13824*x^2 - 427/17915904*x^3 - 7399/123834728448*x^4 - 3647/35664401793024*x^5


In [7]:
print(HsubItoMminus2(5,3))

1 - 7/24*x - 77/13824*x^2 - 427/17915904*x^3 - 7399/123834728448*x^4 - 3647/35664401793024*x^5


In [2]:
print(HsubItoMminus2(5,3))

1 - 7/24*x - 77/13824*x^2 - 427/17915904*x^3 - 7399/123834728448*x^4 - 3647/35664401793024*x^5


In [3]:
import pickle
import time
wfile=open('/Users/barrybrent/6oct22no2.txt','wb')
data=[]
n=200
start=time.time()/60
for m in [3..4803]:
    poly=HsubItoMminus2(n,m)
    data=data+[[m,poly]]
    finish=time.time()/60
    print([m,poly.degree(),finish-start])
t = pickle.dumps(str(data))
wfile.write(t)
wfile.close()

[3, 200, 0.26565640419721603]
[4, 200, 0.44107865169644356]
[5, 200, 0.7939302697777748]
[6, 200, 0.9573802351951599]
[7, 200, 1.347701583057642]
[8, 200, 1.6438663378357887]
[9, 200, 2.0631768852472305]
[10, 200, 2.3295488357543945]
[11, 200, 2.7528453692793846]
[12, 200, 3.116776768118143]
[13, 200, 3.566112168133259]
[14, 200, 3.85887985303998]
[15, 200, 4.361913681030273]
[16, 200, 4.714780367910862]
[17, 200, 5.214596800506115]
[18, 200, 5.5629760548472404]
[19, 200, 6.040303569287062]
[20, 200, 6.466485965996981]
[21, 200, 6.992378819733858]
[22, 200, 7.313118480145931]
[23, 200, 7.798431880772114]
[24, 200, 8.242612317204475]
[25, 200, 8.766115717589855]
[26, 200, 9.131536848843098]
[27, 200, 9.711051400750875]
[28, 200, 10.19984919950366]
[29, 200, 10.731676284223795]
[30, 200, 11.128610014915466]
[31, 200, 11.6404173001647]
[32, 200, 12.09656971693039]
[33, 200, 12.68180625140667]
[34, 200, 13.057926800101995]
[35, 200, 13.65347744897008]
[36, 200, 14.149832498282194]
[37, 200

[276, 200, 161.86296230182052]
[277, 200, 162.5639490634203]
[278, 200, 163.10465420037508]
[279, 200, 163.85352838411927]
[280, 200, 164.55234215036035]
[281, 200, 165.24872986599803]
[282, 200, 165.84035638347268]
[283, 200, 166.55407243221998]
[284, 200, 167.1990916505456]
[285, 200, 168.00241508334875]
[286, 200, 168.5679000504315]
[287, 200, 169.33606024831533]
[288, 200, 169.9786554351449]
[289, 200, 170.73193470016122]
[290, 200, 171.32364286854863]
[291, 200, 172.08018501475453]
[292, 200, 172.7055391818285]
[293, 200, 173.42452308163047]
[294, 200, 174.0301770158112]
[295, 200, 174.78060203418136]
[296, 200, 175.41803643479943]
[297, 200, 176.17522091791034]
[298, 200, 176.70909412950277]
[299, 200, 177.46318328380585]
[300, 200, 178.1655841358006]
[301, 200, 178.92112485319376]
[302, 200, 179.45147813111544]
[303, 200, 180.21643562987447]
[304, 200, 180.86107579991221]
[305, 200, 181.6258903965354]
[306, 200, 182.22323486953974]
[307, 200, 182.95710696652532]
[308, 200, 183.6

[545, 200, 354.9908657670021]
[546, 200, 355.6625716164708]
[547, 200, 356.4145796857774]
[548, 200, 357.12602143362164]
[549, 200, 357.95883875340223]
[550, 200, 358.62674978002906]
[551, 200, 359.4267264381051]
[552, 200, 360.17033441737294]
[553, 200, 360.97096613422036]
[554, 200, 361.57094698399305]
[555, 200, 362.4464872851968]
[556, 200, 363.1309366002679]
[557, 200, 363.9293672181666]
[558, 200, 364.5731989182532]
[559, 200, 365.3919813372195]
[560, 200, 366.13754208385944]
[561, 200, 366.9968237057328]
[562, 200, 367.5997795201838]
[563, 200, 368.375956133008]
[564, 200, 369.1310775987804]
[565, 200, 369.9456504844129]
[566, 200, 370.5698657371104]
[567, 200, 371.425936833024]
[568, 200, 372.1293992511928]
[569, 200, 372.87994385138154]
[570, 200, 373.5655881986022]
[571, 200, 374.3371705636382]
[572, 200, 375.0656112357974]
[573, 200, 375.88910698518157]
[574, 200, 376.5241507180035]
[575, 200, 377.36566848680377]
[576, 200, 378.07932448387146]
[577, 200, 378.8641328662634]
[

[816, 200, 561.7680132836103]
[817, 200, 562.6160923317075]
[818, 200, 563.2708289660513]
[819, 200, 564.1678466200829]
[820, 200, 564.9677572995424]
[821, 200, 565.7764759548008]
[822, 200, 566.4613225832582]
[823, 200, 567.2788331992924]
[824, 200, 568.0256120152771]
[825, 200, 568.939833316952]
[826, 200, 569.6089890375733]
[827, 200, 570.424274802208]
[828, 200, 571.1997846327722]
[829, 200, 572.0136872343719]
[830, 200, 572.6830093488097]
[831, 200, 573.5189695507288]
[832, 200, 574.2478012181818]
[833, 200, 575.1060018017888]
[834, 200, 575.7969910651445]
[835, 200, 576.6256929151714]
[836, 200, 577.4103457853198]
[837, 200, 578.2633999660611]
[838, 200, 578.9110244847834]
[839, 200, 579.7081747837365]
[840, 200, 580.551830869168]
[841, 200, 581.3586803860962]
[842, 200, 581.9770381003618]
[843, 200, 582.8276359848678]
[844, 200, 583.5515810213983]
[845, 200, 584.432211201638]
[846, 200, 585.1094872169197]
[847, 200, 585.9729167670012]
[848, 200, 586.6844674684107]
[849, 200, 587

[1087, 200, 777.4683921486139]
[1088, 200, 778.2282674461603]
[1089, 200, 779.1432725861669]
[1090, 200, 779.8637304157019]
[1091, 200, 780.7084092348814]
[1092, 200, 781.5578468330204]
[1093, 200, 782.3972796164453]
[1094, 200, 783.0598832517862]
[1095, 200, 783.9776695817709]
[1096, 200, 784.7371962331235]
[1097, 200, 785.5898086838424]
[1098, 200, 786.3025127388537]
[1099, 200, 787.1980931013823]
[1100, 200, 788.0394170843065]
[1101, 200, 788.9305525198579]
[1102, 200, 789.6136698536575]
[1103, 200, 790.4565242528915]
[1104, 200, 791.2670000977814]
[1105, 200, 792.1896899491549]
[1106, 200, 792.8996328338981]
[1107, 200, 793.8141200654209]
[1108, 200, 794.6005032323301]
[1109, 200, 795.4620464220643]
[1110, 200, 796.2217937670648]
[1111, 200, 797.1042130328715]
[1112, 200, 797.8701142519712]
[1113, 200, 798.7939636036754]
[1114, 200, 799.4718181155622]
[1115, 200, 800.3787293322384]
[1116, 200, 801.2096179537475]
[1117, 200, 802.0766867510974]
[1118, 200, 802.7660427205265]
[1119, 2

...

[3689, 200, 3082.629282414913]
[3690, 200, 3083.502803515643]
[3691, 200, 3084.490534450859]
[3692, 200, 3085.4084795974195]
[3693, 200, 3086.428818963468]
[3694, 200, 3087.1780753321946]
[3695, 200, 3088.181889615953]
[3696, 200, 3089.143583547324]
[3697, 200, 3090.102724198252]
[3698, 200, 3090.8988199345767]
[3699, 200, 3091.9240636005998]
[3700, 200, 3092.8721880316734]
[3701, 200, 3093.838331449777]
[3702, 200, 3094.662533700466]
[3703, 200, 3095.670521121472]
[3704, 200, 3096.5337021313608]
[3705, 200, 3097.5974242836237]
[3706, 200, 3098.3836967349052]
[3707, 200, 3099.399487283081]
[3708, 200, 3100.3489787355065]
[3709, 200, 3101.332288119942]
[3710, 200, 3102.1884348653257]
[3711, 200, 3103.196950953454]
[3712, 200, 3104.055628914386]
[3713, 200, 3105.046539682895]
[3714, 200, 3105.8590694330633]
[3715, 200, 3106.850679066032]
[3716, 200, 3107.742366217077]
[3717, 200, 3108.801126498729]
[3718, 200, 3109.618180833757]
[3719, 200, 3110.5884781330824]
[3720, 200, 3111.5449532009

[3950, 200, 3327.4744691178203]
[3951, 200, 3328.5028891824186]
[3952, 200, 3329.4308673515916]
[3953, 200, 3330.4040485993028]
[3954, 200, 3331.233060617]
[3955, 200, 3332.279541518539]
[3956, 200, 3333.185990203172]
[3957, 200, 3334.2083433046937]
[3958, 200, 3335.0043436847627]
[3959, 200, 3336.0190589651465]
[3960, 200, 3337.0193662792444]
[3961, 200, 3338.0198266878724]
[3962, 200, 3338.8386716507375]
[3963, 200, 3339.8653927370906]
[3964, 200, 3340.7727720178664]
[3965, 200, 3341.792093668133]
[3966, 200, 3342.6121694520116]
[3967, 200, 3343.5746188536286]
[3968, 200, 3344.4414383992553]
[3969, 200, 3345.4813061505556]
[3970, 200, 3346.321899201721]
[3971, 200, 3347.312622666359]
[3972, 200, 3348.239559933543]
[3973, 200, 3349.2000386193395]
[3974, 200, 3349.9901413694024]
[3975, 200, 3351.0384537391365]
[3976, 200, 3351.9593021832407]
[3977, 200, 3352.9304010495543]
[3978, 200, 3353.7947781160474]
[3979, 200, 3354.795180283487]
[3980, 200, 3355.723634067923]
[3981, 200, 3356.728

[4211, 200, 3574.272280752659]
[4212, 200, 3575.2355570457876]
[4213, 200, 3576.2526131868362]
[4214, 200, 3577.0855958126485]
[4215, 200, 3578.1471917815506]
[4216, 200, 3579.0611235164106]
[4217, 200, 3580.0463689379394]
[4218, 200, 3580.8931793347]
[4219, 200, 3581.863214481622]
[4220, 200, 3582.7989283688366]
[4221, 200, 3583.8446506001055]
[4222, 200, 3584.626254800707]
[4223, 200, 3585.5853913016617]
[4224, 200, 3586.515144266188]
[4225, 200, 3587.5515831001103]
[4226, 200, 3588.348896432668]
[4227, 200, 3589.3720376342535]
[4228, 200, 3590.2809597179294]
[4229, 200, 3591.2524974495173]
[4230, 200, 3592.1266764849424]
[4231, 200, 3593.105574604124]
[4232, 200, 3594.008125703782]
[4233, 200, 3595.0414027646184]
[4234, 200, 3595.8519160859287]
[4235, 200, 3596.918798919767]
[4236, 200, 3597.8663244321942]
[4237, 200, 3598.8549906015396]
[4238, 200, 3599.6857727505267]
[4239, 200, 3600.7090889029205]
[4240, 200, 3601.6453789658844]
[4241, 200, 3602.6298067346215]
[4242, 200, 3603.50

[4471, 200, 3822.889191620052]
[4472, 200, 3823.81418883428]
[4473, 200, 3824.8722790144384]
[4474, 200, 3825.6860074177384]
[4475, 200, 3826.702891431749]
[4476, 200, 3827.66623441875]
[4477, 200, 3828.677146784961]
[4478, 200, 3829.4664919003844]
[4479, 200, 3830.498824700713]
[4480, 200, 3831.4129344560206]
[4481, 200, 3832.407394248992]
[4482, 200, 3833.239700768143]
[4483, 200, 3834.2402213960886]
[4484, 200, 3835.167551100254]
[4485, 200, 3836.2771095372736]
[4486, 200, 3837.081549335271]
[4487, 200, 3838.0937654189765]
[4488, 200, 3839.0892545357347]
[4489, 200, 3840.0748428329825]
[4490, 200, 3840.926923882216]
[4491, 200, 3841.9708139039576]
[4492, 200, 3842.908301651478]
[4493, 200, 3843.9089618995786]
[4494, 200, 3844.782666351646]
[4495, 200, 3845.841122150421]
[4496, 200, 3846.7310556545854]
[4497, 200, 3847.761902552098]
[4498, 200, 3848.5743515491486]
[4499, 200, 3849.620152182877]
[4500, 200, 3850.6076204031706]
[4501, 200, 3851.6553486175835]
[4502, 200, 3852.434152636

[4732, 200, 4073.8285030014813]
[4733, 200, 4074.8543289490044]
[4734, 200, 4075.701336849481]
[4735, 200, 4076.7526070326567]
[4736, 200, 4077.6252712495625]
[4737, 200, 4078.6740274503827]
[4738, 200, 4079.5066249631345]
[4739, 200, 4080.5320089012384]
[4740, 200, 4081.5403805337846]
[4741, 200, 4082.5617313198745]
[4742, 200, 4083.3916098996997]
[4743, 200, 4084.4695411510766]
[4744, 200, 4085.380019851029]
[4745, 200, 4086.4353538677096]
[4746, 200, 4087.3043964877725]
[4747, 200, 4088.2849619202316]
[4748, 200, 4089.2009635679424]
[4749, 200, 4090.233155567199]
[4750, 200, 4091.0780234858394]
[4751, 200, 4092.094000350684]
[4752, 200, 4093.0539788007736]
[4753, 200, 4094.086434982717]
[4754, 200, 4094.883749317378]
[4755, 200, 4095.9549371004105]
[4756, 200, 4096.86055958271]
[4757, 200, 4097.853619668633]
[4758, 200, 4098.732964348048]
[4759, 200, 4099.730993621051]
[4760, 200, 4100.742365736514]
[4761, 200, 4101.794069100171]
[4762, 200, 4102.625123880804]
[4763, 200, 4103.63798

In [ ]:
import pickle
import time
rfile=open('/Users/barrybrent/6oct22no2.txt','rb')
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)


wfile = open('/Users/barrybrent/10oct22.txt','wb') 
#HsubItoMminus2Strike polynomials
polydata=[]
start=time.time()/60
for n in [0..200]:
    data=[]
    for k in [0..len(s)-1]:
        m = s[k][0]
        poly=D(s[k][1])
        cf=polynomialCoefficient(n,poly)
        data=data+[[m,cf]]
    rl=R.lagrange_polynomial(data)
    poly=rl.polynomial(x)
    polydata=polydata+[[n,poly]] 
    finish=time.time()/60
    print([n,poly.degree(),finish-start])
t = pickle.dumps(str(polydata))
wfile.write(t)
wfile.close()#not polynomial? Tty again, omitting k= 0.

[0, 0, 3.0262356773018837]
[1, 4800, 9.249989677220583]
[2, 4800, 38.85746067762375]


In [3]:
import pickle
import time
rfile=open('/Users/barrybrent/6oct22no2.txt','rb')
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)


wfile = open('/Users/barrybrent/10oct22.txt','wb') 
#HsubItoMminus2Strike polynomials
polydata=[]
start=time.time()/60
for n in [1..200]:
    data=[]
    for k in [1..len(s)-1]: #omit k = 0
        m = s[k][0]
        poly=D(s[k][1])
        cf=polynomialCoefficient(n,poly)
        data=data+[[m,cf]]
    rl=R.lagrange_polynomial(data)
    poly=rl.polynomial(x)
    polydata=polydata+[[n,poly]] 
    finish=time.time()/60
    print([n,poly.degree(),finish-start])
t = pickle.dumps(str(polydata))
wfile.write(t)
wfile.close() #still doesn't work

[1, 4799, 6.254186429083347]


KeyboardInterrupt: 

In [2]:
import pickle
rfile = open('/Users/barrybrent/6oct22.txt','rb') #HsubItoMminus2Strike polynomials
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
len(s)

201

In [5]:
import pickle
rfile = open('/Users/barrybrent/6oct22.txt','rb') #HsubItoMminus2Strike polynomials
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
for k in [0..3]:
    n=s[k][0]
    poly=D(s[k][1])
    print("====================================================================================")
    print ((k,n))
    print 
    print (poly)
    print
    print (poly.factor())
    print(numericalfactor(poly))

(0, 0)
1
1
1
(1, 1)
-24*x^4 + 64*x^3 - 32*x^2
(-24) * (x - 2) * (x - 2/3) * x^2
-24
(2, 2)
288*x^8 - 1800*x^7 + 3328*x^6 - 1216*x^5 - 1536*x^4 + 896*x^3
(288) * (x - 2) * (x - 2/3) * (x + 2/3) * x^3 * (x^2 - 17/4*x + 7/2)
288
(3, 3)
-2304*x^12 + 24768*x^11 - 91680*x^10 + 122368*x^9 + 217984/9*x^8 - 177152*x^7 + 752128/9*x^6 + 49152*x^5 - 284672/9*x^4
(-2304) * (x - 2) * (x - 2/3) * x^4 * (x^6 - 97/12*x^5 + 1217/72*x^4 + 74/27*x^3 - 695/27*x^2 + 124/27*x + 278/27)
-2304


In [57]:
import pickle
rfile = open('/Users/barrybrent/6oct22.txt','rb') #HsubItoMminus2Strike polynomials
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
for k in [0..3]:
    n=s[k][0]
    poly=D(s[k][1])
    print("====================================================================================")
    print ((k,n))
    print 
    print (poly)
    print
    print (poly.factor())

n:  0

1
1
n:  1

(-24) * (x - 2/3) * x^2
-24
n:  2

(24) * (x - 14) * (x - 2) * (x - 2/3) * x^3
24
n:  3

(-96) * (x - 2) * (x - 2/3) * x^4 * (x^3 - 50/9*x^2 + 460/9*x - 1112/9)
-96
n:  4

(24) * (x - 2) * (x - 2/3) * x^5 * (x^5 - 298/3*x^4 + 19208/27*x^3 - 90992/27*x^2 + 343600/27*x - 520864/27)
24
n:  5

(-144) * (x - 2) * (x - 2/3) * x^6 * (x^7 - 766/45*x^6 + 633908/2025*x^5 - 5608696/2025*x^4 + 26620592/2025*x^3 - 90708256/2025*x^2 + 45835456/405*x - 268935808/2025)
-144


In [5]:
import pickle
rfile = open('/Users/barrybrent/6oct22.txt','rb') #HsubItoMminus2Strike polynomials
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
old=1
data=[]
for k in [0..len(s)-1]:
    n=s[k][0]
    if n>1:
        poly=D(s[k][1])
        nf=abs(denominator(QQ(numericalfactor(poly))))
        mpd=max(prime_divisors(n))
        twothreedenominator=2^multiplicative_order(2,n)*3^multiplicative_order(3,n)
        skeleton=n/twothreedenominator
        data=data+[nf/old-skeleton]
        old=nf
print(data)

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [47]:
import pickle
rfile = open('/Users/barrybrent/6oct22.txt','rb') #HsubItoMminus2Strike polynomials
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
old=1
data=[]
for k in [0..len(s)-1]:
    n=s[k][0]
    poly=D(s[k][1])
    nf=abs(numerator(QQ(numericalfactor(poly))))
    if n>1:
        mpd=max(prime_divisors(n))
        twothreedenominator=2^multiplicative_order(2,n)*3^multiplicative_order(3,n)
        if twothreedenominator*nf/old!=24:data=data+[n]
    old=nf
print(data)

[]
